## Here I translate GCN in tensorflow to GCN in tensorflow.keras


In [11]:
import tensorflow as tf
import numpy as np
import os
from tensorflow.keras import layers

In [12]:
# tf and tf keras versions
print(tf.VERSION)
print(tf.keras.__version__)

1.13.0-rc2
2.2.4-tf


This class allows us to build the weight and biases for each convolutional layer and also each model. After building the biases in the build method, I will perform the matrix multiplications in the call method for each model. The cell below will iteratively call upon all layers, until the whole model is build. The model will then be compiled. 

In [38]:
adj = np.load('/Users/b_eebs/tf-keras/ZINC/adj/0.npy').astype('float32')
features = np.load('/Users/b_eebs/tf-keras/ZINC/features/0.npy').astype('float32')
test = (np.load('/Users/b_eebs/tf-keras/ZINC/logP.npy')[0:10000]).astype('float32') #takes the first 10,000 molecules

In [75]:
features_ = np.array_split(features,10)
test_ = np.array_split(test,10)
adj_ = np.array_split(adj,10)
print(features_[0].shape, test_[0][0].shape)

(1000, 50, 58) (1000,)


In [77]:
import conv_models
import tensorflow.keras.backend as K

class convLayers(layers.Layer):
    models = ['gcn', 'gcn_a', 'gcn_g', 'gcn_ag']
    

    def __init__(self,adjacency,features,output_dim, 
                 weight_shape,bias_shape, model, **kwargs):
        self.output_dim = output_dim 
        self.weight_shape = weight_shape
        self.bias_shape = bias_shape
        self.model = model
        self.features = K.placeholder(shape = [100,50,58], 
                                       dtype = tf.float32)
        self.adjacency = K.placeholder(shape = [100,50,50], 
                                        dtype = tf.float32)
        super(convLayers, self).__init__(**kwargs)
    
    # still have to add attention weights
    # initializers may be incorrect
    
    def build(self,input_shape):
        self.kernel = self.add_weight(name='kernel',
                                  shape=self.weight_shape,
                                  initializer='glorot_uniform',
                                  trainable=True,dtype=tf.float32)
        
        self.bias = self.add_weight(name='bias',
                                shape=self.bias_shape,
                                initializer='glorot_uniform',
                                trainable=True,dtype=tf.float32)
       
        super(convLayers, self).build(input_shape)
    
    #here is where I will implement all models, which will be built in another script. 
    def call(self, inputs):
        if self.model == convLayers.models[0]:
            print()
            _features = conv_models.GCN(self.features,self.adjacency,
                                        self.kernel,self.bias)
        elif self.model == convLayers.models[1]:
            print('go to gcn+att function and calc. mult.')
        elif self.model == convLayers.models[2]:
            print('go to gcn+gate function and calc. mult.')
        elif self.model == convLayers.models[3]:
            print('go to aug. gcn function and calc. mult.')
        else:
            raise ValueError('not a valid model')
        return _features

    def compute_output_shape(self, input_shape):
        return tf.TensorShape([100,50,32]) 
    

In [78]:
K.set_learning_phase(1)
model = tf.keras.Sequential([convLayers(adj_[0],features_[0],32,[58,32],[32],'gcn'), 
    layers.Activation('relu')])

model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='mse',
              metrics=['mae'])

model.fit(features_[0], test_[0][0], batch_size=100, epochs=5)



Epoch 1/10


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_46' with dtype float and shape [100,50,58]
	 [[{{node Placeholder_46}}]]